# Detección de objetos en video

Este cuaderno implementa un sistema básico de detección y seguimiento de objetos en vídeo utilizando OpenCV y un tracker basado en distancia euclídea.

---

## 1. Objetivo
- Detectar objetos en movimiento dentro de una región de interés (ROI) de un vídeo.  
- Asignar a cada objeto un identificador único que persista mientras permanezca en escena.

---

## 2. Componentes principales
- **Sustracción de fondo (MOG2)**  
  Separa el primer plano (objetos en movimiento) del fondo estático.  
- **Detección de contornos**  
  Extrae las siluetas de cada objeto a partir de la máscara binaria resultante de la sustracción de fondo y un umbral fijo.  
- **Tracker euclidiano**  
  Mantiene un diccionario de centros de objetos y asigna IDs nuevos o existentes según la proximidad de sus posiciones frame a frame.

---

## 3. Flujo de trabajo
1. **Configuración**  
   - Importar librerías.  
   - Definir la clase `EuclideanDistTracker`.  
   - Abrir el vídeo y crear el sustractor de fondo.  
2. **Procesamiento por frame**  
   1. Leer el siguiente frame y recortar la ROI.  
   2. Aplicar sustracción de fondo y umbralización para obtener la máscara binaria.  
   3. Encontrar contornos y filtrar por área mínima.  
   4. Pasar los bounding boxes al tracker para obtener IDs.  
   5. Dibujar rectángulos y etiquetas sobre los objetos en la ROI.  
3. **Visualización**  
   - Mostrar en tiempo real la ROI anotada, el frame completo y la máscara.  
4. **Limpieza**  
   - Cerrar la captura de vídeo y destruir las ventanas de OpenCV.

---

## 4. Extensiones posibles
- Ajustar parámetros de sustracción y filtrado para distintos escenarios de iluminación y tamaños de objeto.  
- Sustituir la detección por un modelo de detección basado en redes neuronales (YOLO, SSD, etc.) para mejorar la precisión.  
- Añadir cálculo de trayectoria o velocidad utilizando los IDs asignados para cada objeto.

En esta celda se importan las dependencias necesarias:
- `cv2` de OpenCV, para procesamiento de vídeo e imágenes.  
- `math`, para cálculos de distancia euclídea.

# Código

In [1]:
# Importat librerías necesarias
import cv2
import math
import numpy as np
import matplotlib.pyplot as plt

Aquí se define la clase encargada de:
- Mantener un diccionario de puntos centrales (`center_points`) de objetos detectados.  
- Asignar un ID único a cada nuevo objeto con `id_count`.  
- El método `update()` recibe una lista de bounding boxes (`[x, y, w, h]`), calcula el centro de cada uno, asocia cada detección con un ID existente (si la distancia entre centros es menor a 25 px) o crea uno nuevo, y devuelve la lista enriquecida con `[x, y, w, h, id]`.

In [2]:
# Definir la clase EuclideanDisTracker para medir la distancia entre los objetos
class EuclideanDistTracker:
    def __init__(self):
        # Almacena los centros de cada objeto detectado
        self.center_points = {}
        # Contador para asignar IDs únicos
        self.id_count = 0

    def update(self, objects_rect):
        """
        Recibe una lista de bounding boxes [x, y, w, h] y devuelve
        la misma lista con un ID asignado a cada objeto: [x, y, w, h, id].
        """
        objects_bbs_ids = []

        # Para cada rectángulo, calcular su centro
        for rect in objects_rect:
            x, y, w, h = rect
            cx = (x + x + w) // 2
            cy = (y + y + h) // 2

            # Buscar si ya tenemos un objeto cercano (distancia < 25 píxeles)
            same_object_detected = False
            for object_id, pt in self.center_points.items():
                dist = math.hypot(cx - pt[0], cy - pt[1])
                if dist < 25:
                    # Actualizar centro y conservar el mismo ID
                    self.center_points[object_id] = (cx, cy)
                    objects_bbs_ids.append([x, y, w, h, object_id])
                    same_object_detected = True
                    break

            # Si no se encontró ninguno cercano, es un objeto nuevo
            if not same_object_detected:
                self.center_points[self.id_count] = (cx, cy)
                objects_bbs_ids.append([x, y, w, h, self.id_count])
                self.id_count += 1

        # Limpiar IDs que ya no están presentes
        new_center_points = {}
        for _, _, _, _, object_id in objects_bbs_ids:
            new_center_points[object_id] = self.center_points[object_id]
        self.center_points = new_center_points.copy()

        return objects_bbs_ids

- Se instancia el tracker (`EuclideanDistTracker`).  
- Se abre el vídeo `highway.mp4` con `cv2.VideoCapture`.  
- Se crea un sustractor de fondo MOG2 (`createBackgroundSubtractorMOG2`) para extraer el foreground en cada frame.

In [4]:
# Configuración de captura de video y detector de fondo
tracker = EuclideanDistTracker()
cap = cv2.VideoCapture("highway.mp4")

# Utilizaremos un detector de fondo tipo MOG2 para extraer el fondo del video
object_detector = cv2.createrBackgroundSubtractorMOG2(history=100, varThreshold=40)

AttributeError: module 'cv2.cv2' has no attribute 'createrBackgroundSubtractorMOG2'

1. **Lectura de cada frame** y comprobación de fin de vídeo.  
2. **Definición de la ROI** donde se procesarán las detecciones.  
3. **Extracción del foreground**:
   - Aplicar el sustractor de fondo.  
   - Umbralizar para obtener una máscara binaria.  
4. **Detección de contornos** en la máscara y filtrado por área mínima (`area > 100`).  
5. **Seguimiento**: pasar las detecciones al tracker, recibir `[x, y, w, h, id]` y dibujar rectángulos y etiquetas con el ID.  
6. **Visualización** de la ROI, el frame completo y la máscara.  
7. **Salida** del bucle al pulsar ESC.

In [ ]:
while True:
    ret, frame = cap.read()
    if not ret:
        break  # Terminar si no hay más frames

    # Recortar región de interés (ROI) donde buscar objetos
    roi = frame[340:720, 500:800]

    # 1) Detección de objetos con sustracción de fondo
    mask = object_detector.apply(roi)
    _, mask = cv2.threshold(mask, 254, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(mask,
                                   cv2.RETR_TREE,
                                   cv2.CHAIN_APPROX_SIMPLE)

    detections = []
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > 100:
            x, y, w, h = cv2.boundingRect(cnt)
            detections.append([x, y, w, h])

    # 2) Seguimiento: asignar IDs a cada detección
    boxes_ids = tracker.update(detections)
    for x, y, w, h, object_id in boxes_ids:
        # Dibujar el ID y el rectángulo en la ROI
        cv2.putText(roi, str(object_id), (x, y - 15),
                    cv2.FONT_HERSHEY_PLAIN, 2, (255, 0, 0), 2)
        cv2.rectangle(roi, (x, y), (x + w, y + h),
                      (0, 255, 0), 3)

    # Mostrar resultados
    cv2.imshow("ROI", roi)
    cv2.imshow("Frame Completo", frame)
    cv2.imshow("Máscara", mask)

    # Salir con la tecla ESC
    if cv2.waitKey(30) == 27:
        break

- Cierra la captura de vídeo con `cap.release()`.  
- Destruye todas las ventanas de OpenCV con `cv2.destroyAllWindows()`.

In [ ]:
# Liberar los recursos cuando ya terminemos
cap.release()
cv2.destroyAllWindows()